# run first iteration tensile test

In [ ]:
from pathlib import Path
import json
import numpy as np
from ase.io import read, write

import main
import dft_engine

ROOT = Path.cwd()
WORKDIR = ROOT / "cases" / "case_center"
pp = ROOT / "al.gga.psp"
assert pp.exists(), f"Missing {pp}"

ref = json.loads((WORKDIR / "ref_energy.json").read_text())
E0 = float(ref["E_ref_eV"])
ecut_ev = float(ref["ecut_ev"])
spacing = float(ref["spacing_A"])

bottom_idx = np.load(WORKDIR / "bottom_idx.npy")
top_idx = np.load(WORKDIR / "top_idx.npy")
fixed_idx = np.unique(np.r_[bottom_idx, top_idx]).astype(int)

cycle = 1
stretched_xyz = WORKDIR / f"cycle_{cycle:03d}_stretched.xyz"
assert stretched_xyz.exists(), f"Missing {stretched_xyz}"

atoms_s = read(str(stretched_xyz))
print("E_ref_eV =", f"{E0:.12f}")
print("N_atoms =", len(atoms_s))

[relax_atoms] before run fixed_z(min/max)=0.000000/40.101306
The final grid size is  [ 78  78 154]
setting key: Al -> /Users/dawson666/Desktop/nano_tensile_TFvW/al.gga.psp
Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       2.763956252217E+03      2.763956E+03    2.794274E+04    1       1       1.964581E+00    
!WARN : pAp small than zero :iter =  1 -3844982.494489998
1       -4.960770921691E+02     -3.260033E+03   1.066435E+03    2       2       2.377450E+00    
2       -7.052921120191E+02     -2.092150E+02   1.766827E+02    3       1       2.805440E+00    
3       -7.387804071218E+02     -3.348830E+01   1.673113E+01    8       2       3.779448E+00    
4       -7.491172416289E+02     -1.033683E+01   7.040636E+00    11      2       4.984565E+00    
5       -7.502657716592E+02     -1.148530E+00   1.509634E+00    22      2       7.198239E+00    
6       -7.504409282386E+02     -1.751566E-01   2.253671E-01    8       2       8.131854E+0

## compare  structure before and after ofdft ionic optimization

In [ ]:
atoms_r, E1, S1 = dft_engine.relax_atoms(
    atoms_s,
    pp_file=pp,
    spacing=spacing,
    fixed_idx=fixed_idx,
    fmax=0.05,
    steps=200,
    logfile=str(WORKDIR / f"cycle_{cycle:03d}_relax.log"),
    trajfile=str(WORKDIR / f"cycle_{cycle:03d}_relax.traj"),
    dftpy_outfile=str(WORKDIR / f"cycle_{cycle:03d}_dftpy.out"),
    debug_fixed=True,
)

relaxed_xyz = WORKDIR / f"cycle_{cycle:03d}_relaxed.xyz"
write(str(relaxed_xyz), atoms_r)

print("E_relaxed_eV =", f"{E1:.12f}")
print("Saved:", relaxed_xyz)

Gauge_L_before_A = 35.424769
Gauge_L_after_A  = 35.424769
dL_relax_A       = 0.000000
Max_disp_all_A   = 3.325274
Max_disp_free_A  = 3.325274


## determine strain energy

In [ ]:
import json

dE = float(E1 - E0)
rec = {
    "cycle": int(cycle),
    "ecut_ev": float(ecut_ev),
    "spacing_A": float(spacing),
    "E_ref_eV": float(E0),
    "E_relaxed_eV": float(E1),
    "dE_eV": float(dE),
}

out_json = WORKDIR / f"cycle_{cycle:03d}_energy.json"
out_json.write_text(json.dumps(rec, indent=2))

print("dE_eV =", f"{dE:.12f}")
print("Saved:", out_json)